# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 23 2023 11:03AM,263479,10,9508519,MedStone Pharma LLC,Released
1,May 23 2023 11:03AM,263479,10,9508520,MedStone Pharma LLC,Released
2,May 23 2023 10:59AM,263478,20,9508516,VITABIOTICS USA INC.,Released
3,May 23 2023 10:59AM,263477,19,021736,"Emersa Waterbox, LLC",Released
4,May 23 2023 10:58AM,263476,19,ELSE52523,Else Nutrition,Released
5,May 23 2023 10:55AM,263475,10,CTF0011765,"Citieffe, Inc.",Released
6,May 23 2023 10:35AM,263473,19,N18092,"NAPP Technologies, LLC",Released
7,May 23 2023 10:34AM,263472,20,9508505,VITABIOTICS USA INC.,Released
8,May 23 2023 10:17AM,263467,21,9508484,"NBTY Global, Inc.",Executing
9,May 23 2023 10:03AM,263465,10,9484332,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,263475,Released,1
29,263476,Released,1
30,263477,Released,1
31,263478,Released,1
32,263479,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
263475,NaN,NaN,1.0
263476,NaN,NaN,1.0
263477,NaN,NaN,1.0
263478,NaN,NaN,1.0
263479,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
263302,0.0,2.0,1.0
263434,7.0,11.0,2.0
263439,0.0,19.0,4.0
263441,0.0,1.0,0.0
263443,3.0,3.0,24.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
263302,0,2,1
263434,7,11,2
263439,0,19,4
263441,0,1,0
263443,3,3,24


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,263302,0,2,1
1,263434,7,11,2
2,263439,0,19,4
3,263441,0,1,0
4,263443,3,3,24


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,263302,,2,1
1,263434,7,11,2
2,263439,,19,4
3,263441,,1,
4,263443,3,3,24


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 23 2023 11:03AM,263479,10,MedStone Pharma LLC
2,May 23 2023 10:59AM,263478,20,VITABIOTICS USA INC.
3,May 23 2023 10:59AM,263477,19,"Emersa Waterbox, LLC"
4,May 23 2023 10:58AM,263476,19,Else Nutrition
5,May 23 2023 10:55AM,263475,10,"Citieffe, Inc."
6,May 23 2023 10:35AM,263473,19,"NAPP Technologies, LLC"
7,May 23 2023 10:34AM,263472,20,VITABIOTICS USA INC.
8,May 23 2023 10:17AM,263467,21,"NBTY Global, Inc."
9,May 23 2023 10:03AM,263465,10,Eywa Pharma Inc.
12,May 23 2023 9:47AM,263302,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 23 2023 11:03AM,263479,10,MedStone Pharma LLC,,,2
1,May 23 2023 10:59AM,263478,20,VITABIOTICS USA INC.,,,1
2,May 23 2023 10:59AM,263477,19,"Emersa Waterbox, LLC",,,1
3,May 23 2023 10:58AM,263476,19,Else Nutrition,,,1
4,May 23 2023 10:55AM,263475,10,"Citieffe, Inc.",,,1
5,May 23 2023 10:35AM,263473,19,"NAPP Technologies, LLC",,,1
6,May 23 2023 10:34AM,263472,20,VITABIOTICS USA INC.,,,1
7,May 23 2023 10:17AM,263467,21,"NBTY Global, Inc.",,1,
8,May 23 2023 10:03AM,263465,10,Eywa Pharma Inc.,,,3
9,May 23 2023 9:47AM,263302,15,"Alliance Pharma, Inc.",,2,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2023 11:03AM,263479,10,MedStone Pharma LLC,2,,
1,May 23 2023 10:59AM,263478,20,VITABIOTICS USA INC.,1,,
2,May 23 2023 10:59AM,263477,19,"Emersa Waterbox, LLC",1,,
3,May 23 2023 10:58AM,263476,19,Else Nutrition,1,,
4,May 23 2023 10:55AM,263475,10,"Citieffe, Inc.",1,,
5,May 23 2023 10:35AM,263473,19,"NAPP Technologies, LLC",1,,
6,May 23 2023 10:34AM,263472,20,VITABIOTICS USA INC.,1,,
7,May 23 2023 10:17AM,263467,21,"NBTY Global, Inc.",,1,
8,May 23 2023 10:03AM,263465,10,Eywa Pharma Inc.,3,,
9,May 23 2023 9:47AM,263302,15,"Alliance Pharma, Inc.",1,2,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2023 11:03AM,263479,10,MedStone Pharma LLC,2,,
1,May 23 2023 10:59AM,263478,20,VITABIOTICS USA INC.,1,,
2,May 23 2023 10:59AM,263477,19,"Emersa Waterbox, LLC",1,,
3,May 23 2023 10:58AM,263476,19,Else Nutrition,1,,
4,May 23 2023 10:55AM,263475,10,"Citieffe, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2023 11:03AM,263479,10,MedStone Pharma LLC,2.0,NaN,NaN
1,May 23 2023 10:59AM,263478,20,VITABIOTICS USA INC.,1.0,NaN,NaN
2,May 23 2023 10:59AM,263477,19,"Emersa Waterbox, LLC",1.0,NaN,NaN
3,May 23 2023 10:58AM,263476,19,Else Nutrition,1.0,NaN,NaN
4,May 23 2023 10:55AM,263475,10,"Citieffe, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 23 2023 11:03AM,263479,10,MedStone Pharma LLC,2.0,0.0,0.0
1,May 23 2023 10:59AM,263478,20,VITABIOTICS USA INC.,1.0,0.0,0.0
2,May 23 2023 10:59AM,263477,19,"Emersa Waterbox, LLC",1.0,0.0,0.0
3,May 23 2023 10:58AM,263476,19,Else Nutrition,1.0,0.0,0.0
4,May 23 2023 10:55AM,263475,10,"Citieffe, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2634615,26.0,1.0,0.0
15,263302,1.0,2.0,0.0
19,2107653,44.0,56.0,3.0
20,1053840,6.0,27.0,7.0
21,263467,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2634615,26.0,1.0,0.0
1,15,263302,1.0,2.0,0.0
2,19,2107653,44.0,56.0,3.0
3,20,1053840,6.0,27.0,7.0
4,21,263467,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,26.0,1.0,0.0
1,15,1.0,2.0,0.0
2,19,44.0,56.0,3.0
3,20,6.0,27.0,7.0
4,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,26.0
1,15,Released,1.0
2,19,Released,44.0
3,20,Released,6.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21
Status,,,,,
Completed,0.0,0.0,3.0,7.0,0.0
Executing,1.0,2.0,56.0,27.0,1.0
Released,26.0,1.0,44.0,6.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21
0,Completed,0.0,0.0,3.0,7.0,0.0
1,Executing,1.0,2.0,56.0,27.0,1.0
2,Released,26.0,1.0,44.0,6.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21
0,Completed,0.0,0.0,3.0,7.0,0.0
1,Executing,1.0,2.0,56.0,27.0,1.0
2,Released,26.0,1.0,44.0,6.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()